# Bibliotecas

In [ ]:
import random
import heapq
import random

NameError: name 'seed' is not defined

# Leitura de dados

In [14]:
def read_instances(file_path):
    instances = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.isdigit():
                size = int(line)
                i += 1
                while i < len(lines):
                    if lines[i].strip() == '':
                        i += 1
                        if i < len(lines) and lines[i].strip() == '':
                            i += 1
                            break  # Duas linhas vazias consecutivas, mudar o tamanho da instância
                        continue  # Linha vazia, continuar para a próxima linha
                    numbers = []
                    for _ in range(size):
                        if i < len(lines):
                            num_line = lines[i].strip()
                            if num_line:
                                numbers.append(int(num_line))
                            i += 1
                    instances.append(numbers)
            else:
                i += 1  # Pula linhas vazias ou não numéricas
    return instances

# Exemplo de uso
file_path = 'dados_literatura_Particao_Numeros.txt'  # Ajustar o caminho conforme necessário
instances = read_instances(file_path)

# Verificando a leitura das instâncias
for idx, instance in enumerate(instances):
    print(f"Instância {idx+1} ({len(instance)} números): {instance}")

Instância 1 (15 números): [1860593326, 5956064568, 8784577856, 4894035213, 716642272, 9233584296, 2338836548, 1220884556, 6222901391, 9764627608, 109435896, 854909134, 477886997, 1743072596, 3641281477]
Instância 2 (15 números): [5128991022, 4532281013, 2537593172, 771452685, 5415152717, 4872884847, 4930454686, 9046553222, 5129738035, 1440703686, 8548266266, 9100734872, 9441006496, 491294914, 9256635732]
Instância 3 (15 números): [9496327829, 3008335560, 3182609501, 8459047958, 3244364330, 9321283309, 7532073923, 6891220098, 4849464750, 1393776662, 7994090908, 8386779510, 9206324775, 7251315129, 3950961279]
Instância 4 (15 números): [1854855597, 3785471115, 5966617027, 4146562220, 1063583983, 6769583879, 9132701079, 4716977964, 4666090587, 4076652849, 8441094368, 7454624448, 8279474265, 4189737317, 875115524]
Instância 5 (15 números): [5132461393, 2241395660, 6711623464, 2825959894, 6773093408, 2216884340, 2734329117, 532544630, 1386706201, 4942648847, 6089827028, 3820669186, 824579466

# BRKGA

In [ ]:
# Função para decodificar um vetor de chaves aleatórias em uma solução de partição
def decode_solution(random_keys, numbers):
    subset1 = []  # Inicializa o primeiro subconjunto
    subset2 = []  # Inicializa o segundo subconjunto
    for i, key in enumerate(random_keys):
        if key < 0.5:  # Se a chave é menor que 0.5
            subset1.append(numbers[i])  # Adiciona o número ao subset1
        else:
            subset2.append(numbers[i])  # Caso contrário, adiciona ao subset2
    return subset1, subset2  # Retorna os dois subconjuntos

# Função para avaliar uma solução de partição
def evaluate_solution(subset1, subset2):
    return abs(sum(subset1) - sum(subset2))  # Calcula e retorna a diferença absoluta entre as somas dos subconjuntos

# Função para gerar uma população inicial de vetores de chaves aleatórias
def generate_initial_population(pop_size, num_elements):
    return [[random.random() for _ in range(num_elements)] for _ in range(pop_size)]  
    # Gera uma população de vetores aleatórios de tamanho pop_size, cada vetor com num_elements elementos

# Função para realizar cruzamento enviesado
def biased_crossover(parent1, parent2, bias=0.7):
    child = []  # Inicializa o vetor filho
    for i in range(len(parent1)):
        if random.random() < bias:
            child.append(parent1[i])  # Adiciona o gene do parent1 com probabilidade 'bias'
        else:
            child.append(parent2[i])  # Caso contrário, adiciona o gene do parent2
    return child  # Retorna o filho gerado

# Função para aplicar mutação em um indivíduo
def mutate(individual, mutation_rate=0.1):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random.random()  # Substitui o gene por um valor aleatório com a taxa de mutação
    return individual  # Retorna o indivíduo mutado

# Função principal do BRKGA
def brkga(numbers, pop_size=100, num_generations=100, elite_fraction=0.2, mutation_rate=0.1, seed = 1290):
    random.seed(seed)
    num_elements = len(numbers)  # Número de elementos no problema
    population = generate_initial_population(pop_size, num_elements)  # Gera a população inicial
    elite_size = int(pop_size * elite_fraction)  # Calcula o tamanho da elite
    
    for generation in range(num_generations):
        # Decodificar e avaliar a população
        decoded_population = [decode_solution(ind, numbers) for ind in population]  # Decodifica cada indivíduo
        evaluated_population = [(evaluate_solution(sub1, sub2), ind) for (sub1, sub2), ind in zip(decoded_population, population)]
        # Avalia cada solução decodificada
        evaluated_population.sort(key=lambda x: x[0])  # Ordena a população avaliada pela diferença absoluta
        
        # Selecionar a elite
        elite = [ind for _, ind in evaluated_population[:elite_size]]  # Seleciona os indivíduos da elite
        
        # Gerar nova população
        new_population = elite[:]  # Inicia a nova população com a elite
        while len(new_population) < pop_size:
            parent1 = random.choice(elite)  # Escolhe um pai da elite
            parent2 = random.choice(population)  # Escolhe um pai da população geral
            child = biased_crossover(parent1, parent2)  # Realiza o cruzamento enviesado
            child = mutate(child, mutation_rate)  # Aplica a mutação
            new_population.append(child)  # Adiciona o filho à nova população
        
        population = new_population  # Atualiza a população
    
    # Retornar a melhor solução encontrada
    best_solution = min(evaluated_population, key=lambda x: x[0])  # Encontra a melhor solução na população avaliada
    return decode_solution(best_solution[1], numbers), best_solution[0]  # Retorna a melhor partição e sua avaliação

# Aplicando BRKGA para cada instância lida
for idx, instance in enumerate(instances):
    best_partition, best_cost = brkga(instance)  # Aplica BRKGA na instância atual
    print(f"Instância {idx+1} ({len(instance)} números):")
    print(f"Melhor partição: {best_partition}")  # Imprime a melhor partição encontrada
    print(f"Diferença mínima entre os subconjuntos: {best_cost}\n")  # Imprime a diferença mínima entre os subconjuntos


Diferença mínima entre os subconjuntos: 2043812

Diferença mínima entre os subconjuntos: 6137083

Diferença mínima entre os subconjuntos: 104944973

Diferença mínima entre os subconjuntos: 21246988

Diferença mínima entre os subconjuntos: 36172

Diferença mínima entre os subconjuntos: 3042576

Diferença mínima entre os subconjuntos: 230718

Diferença mínima entre os subconjuntos: 1149654

Diferença mínima entre os subconjuntos: 186530

Diferença mínima entre os subconjuntos: 253021

Diferença mínima entre os subconjuntos: 2834898

Diferença mínima entre os subconjuntos: 4563268

Diferença mínima entre os subconjuntos: 3816424

Diferença mínima entre os subconjuntos: 114418

Diferença mínima entre os subconjuntos: 10099761

Diferença mínima entre os subconjuntos: 8051882

Diferença mínima entre os subconjuntos: 10635981

Diferença mínima entre os subconjuntos: 1955035

Diferença mínima entre os subconjuntos: 7308847

Diferença mínima entre os subconjuntos: 70927

Diferença mínima entre 

# Grasp com Karmarkar-Karp

In [ ]:

# Função para aplicar o algoritmo de Karmarkar-Karp em uma lista de números
def karmarkar_karp(numbers):
    heap = [-num for num in numbers]  # Converte os números para negativos e cria uma heap
    heapq.heapify(heap)  # Constrói uma heap a partir da lista de números negativos
    
    while len(heap) > 1:
        largest = -heapq.heappop(heap)  # Remove o maior elemento da heap
        second_largest = -heapq.heappop(heap)  # Remove o segundo maior elemento da heap
        difference = largest - second_largest  # Calcula a diferença entre os dois maiores elementos
        heapq.heappush(heap, -difference)  # Adiciona a diferença de volta à heap
    
    return -heap[0]  # Retorna o valor absoluto do último elemento restante na heap

# Função para construção gulosa randomizada de uma solução
def greedy_randomized_construction(numbers, alpha=0.3):
    # Ordena os números em ordem decrescente
    sorted_numbers = sorted(numbers, reverse=True)
    solution = []
    
    while sorted_numbers:
        # Seleciona um subconjunto de candidatos
        candidates = sorted_numbers[:int(alpha * len(sorted_numbers)) + 1]
        # Escolhe um candidato aleatoriamente
        selected = random.choice(candidates)
        solution.append(selected)
        sorted_numbers.remove(selected)
    
    return solution

# Função para aplicar a busca local, utilizando o algoritmo de Karmarkar-Karp
def local_search(numbers):
    return karmarkar_karp(numbers)  # Aplica o algoritmo de Karmarkar-Karp para melhorar a solução

# Função principal do GRASP
def grasp(numbers, max_iterations=100, alpha=0.3, seed = 120):
    random.seed(seed)
    best_solution = None
    best_difference = float('inf')  # Inicializa a melhor diferença como infinito
    
    for _ in range(max_iterations):
        # Construção
        constructed_solution = greedy_randomized_construction(numbers, alpha)
        
        # Busca local
        current_difference = local_search(constructed_solution)
        
        # Atualiza a melhor solução
        if current_difference < best_difference:
            best_difference = current_difference
            best_solution = constructed_solution[:]
    
    return best_difference, best_solution  # Retorna a melhor diferença e a melhor solução

# Aplicando o GRASP com Karmarkar-Karp em cada instância
for idx, numbers in enumerate(instances):
    min_difference, best_solution = grasp(numbers)  # Aplica o GRASP na instância atual
    print(f"Instância {idx+1} ({len(numbers)} números):")
    print(f"Melhor solução: {best_solution}")  # Imprime a melhor solução encontrada
    print(f"Diferença mínima entre os subconjuntos: {min_difference}\n")  # Imprime a diferença mínima entre os subconjuntos


Diferença mínima entre os subconjuntos: 2043812

Diferença mínima entre os subconjuntos: 49752055

Diferença mínima entre os subconjuntos: 14440579

Diferença mínima entre os subconjuntos: 244401558

Diferença mínima entre os subconjuntos: 11307616

Diferença mínima entre os subconjuntos: 24124

Diferença mínima entre os subconjuntos: 537372

Diferença mínima entre os subconjuntos: 195668

Diferença mínima entre os subconjuntos: 16408

Diferença mínima entre os subconjuntos: 719839

Diferença mínima entre os subconjuntos: 135442

Diferença mínima entre os subconjuntos: 67872

Diferença mínima entre os subconjuntos: 21302

Diferença mínima entre os subconjuntos: 332444

Diferença mínima entre os subconjuntos: 28187

Diferença mínima entre os subconjuntos: 172

Diferença mínima entre os subconjuntos: 3291

Diferença mínima entre os subconjuntos: 11705

Diferença mínima entre os subconjuntos: 110327

Diferença mínima entre os subconjuntos: 15327

Diferença mínima entre os subconjuntos: 31